# Sagemaker Donkey neural network

It's time to dig into the neural network code.

The goals of this chapter are:
- Get familiar with the neural network(s) used in the [Donkey](https://github.com/wroscoe/donkey) library
- Make some minor changes to the network and
- Create a custom car based on the `donkey/donkeycar/templates/donkey2.py` template

If you're running this lab as a part of a workshop, now is a good time to ask the instructor to hold the [ML](../presentations/ai.md) presentation.

## Inspecting the Keras network

The following code assumes that the [Donkey](https://github.com/wroscoe/donkey) library is properly installed, see [previous chapter](./donkey-train.ipynb#install).

First, take a few minutes to look through the `keras.py` file (it's a [Donkey](https://github.com/wroscoe/donkey) library *part*):

In [ ]:
# Assuming donkey library is installed in ./donkey
%pycat donkey/donkeycar/parts/keras.py

The default algorithm used is defined in `donkey/donkeycar/templates/donkey2.py`:

```python
def drive(cfg, model_path=None, use_joystick=False):
    ...
    kl = KerasCategorical()
    if model_path:
        kl.load(model_path)
    ...
```

`KerasCategorical`, which is created in `default_categorical()` method. Before looking closer at the code, let's see if we can visualize it using our pre-trained model (see [previous](./donkey-train.ipynb) chapter):

In [ ]:
# Download the model. Replace bucket if you used some other bucket than SageMaker default
import sagemaker
bucket = sagemaker.Session().default_bucket()
!aws s3 cp s3://{bucket}/models/my-first-model .

In [ ]:
# Install some additional python libraries required by the Keras visualization lib
!conda install --yes pydot graphviz

In [ ]:
# Plot the algorithm
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.models import load_model

model = load_model('./my-first-model')
SVG(model_to_dot(model).create(prog='dot', format='svg'))

Ah! A nice neural network with
- 1 input layer
- 5 convolutional layers
- 1 flatten layer
- 2 dense layers
- 2 dropout layers

The presence of a convolutional layer makes this neural network a [convolutional network](https://en.wikipedia.org/wiki/Convolutional_neural_network). This makes sence, since CNNs are particulary good with images.

So, what do the different layers actually do? Let's have a look at them, one at a time.

#### Quick readup

This excellent cheatsheet is handy to read through and use as a reference before continuing:
- http://ml-cheatsheet.readthedocs.io/en/latest/nn_concepts.html

And two links about CNNs (read, or be confused later):
- https://adeshpande3.github.io/adeshpande3.github.io/A-Beginner's-Guide-To-Understanding-Convolutional-Neural-Networks/
- https://cambridgespark.com/content/tutorials/convolutional-neural-networks-with-keras/index.html

#### Input layer

The *Input layer* holds the input data.

In [ ]:
from keras.layers import Input

# Input layer
#
# 120 x 160 x 3 image size (RGB)
#
img_in = Input(shape=(120, 160, 3), name='img_in')

# Rename it to x
x = img_in

We define the shape of the data as a 3 channel (RGB) 120px x 160px image.

If you read through the [links](#quick-readup), you'll have a rought idea of what a channel is. In CNNs,  layers operate on 3D chunks of data. The first two dimensions are the height and width of the image, and the third dimension is a number of such 2D data stacked over each other (3 in RGB images). This stack is called channels.

Why is this important? In the input layer, a channel is easily understood as RGB data (one channel per color). However, a [convolutional layer](#first-convolutional-layer) can have many more channels, which we'll get to later.

Formally, x can be viewed as a $H \times W \times C$ vector, where `H,W` are the dimensions of the image and `C` is the number of channels of the given image volume.

API documentation:
- https://keras.io/layers/core/#input

#### First convolutional layer

Before continuing, make sure you've read through and have a rough idea of the following concepts:
- NN concepts - http://ml-cheatsheet.readthedocs.io/en/latest/nn_concepts.html
- Relu - http://ml-cheatsheet.readthedocs.io/en/latest/activation_functions.html#relu
- Convolution operator - https://cambridgespark.com/content/tutorials/convolutional-neural-networks-with-keras/index.html#convolutions

In [ ]:
from keras.layers import Convolution2D

# First hidden layer
#
# 24 features      - Results in a 24 channel feature map(s). This means the first layer can detect 24 different low level features in the image.
# 5x5 pixel kernel - Width and height of the kernel. Will automatically have the same channel depth as the input (5x5x3)
# 2w x 2h stride   - Strides of the convolution along the width and height
# relu activation  - Use the 'relu' activation function
#
x = Convolution2D(24, (5,5), strides=(2,2), activation='relu')(x)

The first hidden layer is a 2D convolution layer with the described hyperparameters (see above) and a `relu` activation function.

> What is the reasoning behind this design? Why are the hyperparameters given these values?

This is a tricky question to answer, but important for later tweaking of the network. The sad news are that they often required lots of experience and theoretical background to master. But hey, let's give it a shot!

They ar all a tradeoff between performance and accuracy, but there are some general rules to follow. For example, the number of *features* are usually lower in the first convolutional layers, because the input size is still large, resulting in large *feature maps*. Later layers have smaller (but deeper) input size (because of the convolution in the previous layers), and thus can afford to have more features. Similar reasoning can be done for the other hyperparameters. The following link has a more in depth discussion around CNN hyperparameters:
- http://deeplearning.net/tutorial/lenet.html#tips-and-tricks

API documentation:
- https://keras.io/layers/convolutional/#conv2d

#### Second convolutional layer

In [ ]:
# Second hidden layer
#
# 32 features      - Results in a 32 channel feature map(s)
#
x = Convolution2D(32, (5,5), strides=(2,2), activation='relu')(x)

Not much changed here. We increase the number of features as discussed earlier. This allows the network to pick up more features based on the feature maps from the first hidden layer.

#### Third convolutional layer

In [ ]:
# Third hidden layer
#
# 64 features      - Results in a 64 channel feature map(s)
#
x = Convolution2D(64, (5,5), strides=(2,2), activation='relu')(x)

We increase the number of features yet again. The input to the next layer will now be 64 channels deep.

#### Fourth convolutional layer

In [ ]:
# Fourth hidden layer
#
# 3x3 pixel kernel   - Width and height of the kernel. Will automatically have the same channel depth as the input (3x3x64)
x = Convolution2D(64, (3,3), strides=(2,2), activation='relu')(x)

Here we decrease the kernel size to 3x3. By using a relatively large kernel size in the 3 previous layers, we have shrunk the image size by quite a bit (each convolution will shrink the image/feature map (if not padded), and add depth instead). By decreasing the kernel size, we make sure that image doesn't shrink too much.

#### Fifth convolutional layer

In [ ]:
# Fifth hidden layer
#
# 1w x 1h stride   -
x = Convolution2D(64, (3,3), strides=(1,1), activation='relu')(x)

In the last convolution layer, we change the stride length. This will result in larger feature maps (at the cost of performance).

#### Flatten layer

In [ ]:
from keras.layers import Flatten 

# Flatten layer
#
# Flatten to 1 dimension
#
x = Flatten(name='flattened')(x)

A flatten layer does what it claims to do. It flattens the convoluted input from 64 channels to 1, by putting the channels after each other. This is needed for the next layer, a fully-connected MLP (Dense).

API documentation:
- https://keras.io/layers/core/#flatten

#### First dense layer

In [ ]:
from keras.layers import Dense

# First Dense layer
#
# 100 units       - Use 100 neurons in the layer
# relu activation - Use 'relu' activation
#
x = Dense(100, activation='relu')(x)

A dense layer is another name for a fully-connected layer like in a [MLP](https://en.wikipedia.org/wiki/Multilayer_perceptron). It is very common for upper-layers in a CNNs to have fully-connected layers (actually, the purpose of conv layers is to extract important features from the image before downsampling enough to be handled by a MLP).

#### First dropout

In [ ]:
from keras.layers import Dropout

# First dropout
#
# 0.1 - Randomly drop out 10% of the neurons to prevent overfitting.
#
x = Dropout(.1)(x)

As described in the [links](), dropout is used to prevent overfitting by forcing redundancy in the neural network (which means it cannot rely on a particular neuron because it might be dropped).

#### Second dense layer

In [ ]:
# Second Dense layer
#
# 50 units        - Use 50 neurons in the layer
#
x = Dense(50, activation='relu')(x)

We gradually decrease the size of the layers from the original 100 to 50.

#### Second dropout

In [ ]:
# Second dropout
#
# Not much to say here...
#
x = Dropout(.1)(x)

#### Output layer

In [ ]:
# Outputs
#
# Angle
# Dense              - Fully-connected output layer
# 15 units           - Use a 15 neuron output.
# softmax activation - Use 'softmax' activation
#
# Throttle
# Dense           - Fully-connected output layer
# 1 unit          - Use a 1 neuron output
# relu activation - Use 'relu' activation
angle_out = Dense(15, activation='softmax', name='angle_out')(x)
throttle_out = Dense(1, activation='relu', name='throttle_out')(x)

The output layers are fully-connected with different units and activations.

Angle uses a 15 neuron output with a softmax activation. [Softmax](https://en.wikipedia.org/wiki/Softmax_function) is a common way of creating a probability distribution over K different possible outcomes (in this case, `K=15`).

Throttle uses a 1 neuron output with a relu activation. [Relu](https://en.wikipedia.org/wiki/Rectifier_(neural_networks) will result in the throttle having only positive values (it can only go forward).

#### Create the model

Finally, it's time to create the model and define the loss functions for training.

In [ ]:
from keras.models import Model

# Create model
#
# Optimizer
# ---------
# adam
#
# Angle
# -----
# categorical cross entropy loss function
# 0.9 loss weight
#
# Throttle
# --------
# mean_absolute_error loss function
# 0.001 loss weight
# 
model = Model(inputs=[img_in], outputs=[angle_out, throttle_out])
model.compile(
    optimizer='adam',
    loss={'angle_out': 'categorical_crossentropy', 'throttle_out': 'mean_absolute_error'},
    loss_weights={'angle_out': 0.9, 'throttle_out': .001})

The model requires the following information:
- input (`img_in`) and output tensors (`angle_out`, `throttle_out`)
- optimizer ([`adam`](https://en.wikipedia.org/wiki/Stochastic_gradient_descent#Adam)). The optimizer is the algorithm used when minimizing the loss function during training. Performs well and efficient.
- loss function(s), one for each output
  - `angle_out` : [`categorial_crossentropy`](https://en.wikipedia.org/wiki/Cross_entropy) - Suitable for categorization.
  - `throttle_out` : [`mean_absolute_error`](https://en.wikipedia.org/wiki/Mean_absolute_error) - More general loss function
- initial loss weights

## Creating a Donkey vehicle

Puh, that was a lot of one-way information. So, let's actually code something by creating our own `DonkeyCar` with a slightly changed `keras.py`.

As mentioned in [earlier chapters](./donkey-tools.ipynb), the [Donkey library](https://github.com/wroscoe/donkey) has several components:
- A python library installed in your preferred location (e.g. system python or virtualenv). You `import` it as any other python library, e.g: `import donkeycar as dk`.
- A CLI with tools mainly used to aid training (see [donkey-tools.ipynb](./donkey-tools.ipynb))
- A `Vehicle` installation, usually installed to the `~/d2` directory. This is where the `manage.py` script is installed, which is used for both **driving** and **training**. The `~/d2` directory and driving the car is discussed further in the [Drive Track](../README.md#the-drive-track).

Let's, however, have a look at the **training** part of a Donkey `Vehicle` and `manage.py`. In short, this is what's going on:
1. When you install the `Vehicle` to the `~/d2` directory, it is per default installed from the [`donkeycar/templates/donkey2.py`](https://github.com/wroscoe/donkey/blob/master/donkeycar/templates/donkey2.py) template. It creates a `Vehicle` using different Donkey parts.
2. Creating `Donkey` parts is more of a focus in the [IoT track](../docs/PREPARE-IOT.md), so we will not dig too deep into what is going on (we leave that for you to sync within the team =)). Examples of parts are Joystick, motor driver and the ML model. See github for more parts: https://github.com/wroscoe/donkey/tree/master/donkeycar/parts
3. The installed `Vehicle` has 2 standard methods; `drive()` and `train()` and is invoked using the CLI. 
  - `drive()` can either be set to manual, and produce training data, or be given a pre-trained model to use when driving autonomously.
  - `train()` trains the model using the collected training data (*Tubs*). Usually done in SageMaker (see [previous chapter](./donkey-train.ipynb)) or on the host computer, which requires you to install the library there first.
  
In this chapter, we'll create a new, simple template for training only, and install it on the training machine (SageMaker or host).

### Create a new Donkey part

First, we'll create a copy of the model used in training as a new `Donkey` part. Create a file called `cnn.py` in the `donkeycar/parts/` by pasting the following (assuming you have `donkey` in you local directory):

In [ ]:
%%bash
# Create a new part. Assumes donkey/ is in current directory
cat > donkey/donkeycar/parts/cnn.py << EOF
import keras
import donkeycar as dk


class KerasPilot():

    def load(self, model_path):
        self.model = keras.models.load_model(model_path)

    def shutdown(self):
        pass
    
    def train(self, train_gen, val_gen, 
              saved_model_path, epochs=100, steps=100, train_split=0.8,
              verbose=1, min_delta=.0005, patience=5, use_early_stop=True):

        save_best = keras.callbacks.ModelCheckpoint(saved_model_path, 
                                                    monitor='val_loss', 
                                                    verbose=verbose, 
                                                    save_best_only=True, 
                                                    mode='min')
        
        early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                   min_delta=min_delta, 
                                                   patience=patience, 
                                                   verbose=verbose, 
                                                   mode='auto')

        callbacks_list = [save_best]

        if use_early_stop:
            callbacks_list.append(early_stop)
        
        hist = self.model.fit_generator(
                        train_gen, 
                        steps_per_epoch=steps, 
                        epochs=epochs, 
                        verbose=1, 
                        validation_data=val_gen,
                        callbacks=callbacks_list, 
                        validation_steps=steps*(1.0 - train_split))
        return hist


class MyPilot(KerasPilot):
    def __init__(self, model=None, *args, **kwargs):
        super(MyPilot, self).__init__(*args, **kwargs)
        self.model = model if model else default_categorical()
        
    def run(self, img_arr):
        img_arr = img_arr.reshape((1,) + img_arr.shape)
        angle_binned, throttle = self.model.predict(img_arr)
        angle_unbinned = dk.utils.linear_unbin(angle_binned)
        return angle_unbinned, throttle[0][0]


def default_categorical():
    from keras.layers import Input, Dense, merge
    from keras.models import Model
    from keras.layers import Convolution2D, MaxPooling2D, Reshape, BatchNormalization
    from keras.layers import Activation, Dropout, Flatten, Dense
    
    img_in = Input(shape=(120, 160, 3), name='img_in')
    x = img_in
    x = Convolution2D(24, (5,5), strides=(2,2), activation='relu')(x)
    x = Convolution2D(32, (5,5), strides=(2,2), activation='relu')(x)
    x = Convolution2D(64, (5,5), strides=(2,2), activation='relu')(x)
    x = Convolution2D(64, (3,3), strides=(2,2), activation='relu')(x)
    x = Convolution2D(64, (3,3), strides=(1,1), activation='relu')(x)

    x = Flatten(name='flattened')(x)
    x = Dense(100, activation='relu')(x)
    x = Dropout(.1)(x)
    x = Dense(50, activation='relu')(x)
    x = Dropout(.1)(x)
    angle_out = Dense(15, activation='softmax', name='angle_out')(x)
    throttle_out = Dense(1, activation='relu', name='throttle_out')(x)
    
    model = Model(inputs=[img_in], outputs=[angle_out, throttle_out])
    model.compile(optimizer='adam',
                  loss={'angle_out': 'categorical_crossentropy', 
                        'throttle_out': 'mean_absolute_error'},
                  loss_weights={'angle_out': 0.9, 'throttle_out': .001})

    return model

EOF

Nice. As you can see, we've basically just removed some code from `keras.py`. Reinstall the library:

In [ ]:
# Replace the path with path to where you cloned the donkey git 
!pip install -e donkey

Create a new simple `Vechicle` template used only for training, but tell it to only run 1 epoch (this is only for testing, we don't want to wait for training to finish...):

In [ ]:
%%bash
# Create new Donkey template. Assuming donkey/ is in current directory
cat > donkey/donkeycar/templates/robolab.py << EOF
#!/usr/bin/env python3
"""
Scripts to train a donkey 2.

Usage:
    manage.py (train) [--tub=<tub1,tub2,..tubn>]  (--model=<model>)

Options:
    -h --help        Show this screen.
    --tub TUBPATHS   List of paths to tubs. Comma separated. Use quotes to use wildcards. ie "~/tubs/*"
"""
import os
from docopt import docopt
import donkeycar as dk

from donkeycar.parts.cnn import MyPilot
from donkeycar.parts.datastore import TubGroup


def train(cfg, tub_names, model_name):
    X_keys = ['cam/image_array']
    y_keys = ['user/angle', 'user/throttle']

    def rt(record):
        record['user/angle'] = dk.utils.linear_bin(record['user/angle'])
        return record

    kl = MyPilot()
    print('tub_names', tub_names)
    if not tub_names:
        tub_names = os.path.join(cfg.DATA_PATH, '*')
    tubgroup = TubGroup(tub_names)
    train_gen, val_gen = tubgroup.get_train_val_gen(X_keys, y_keys, record_transform=rt,
                                                    batch_size=cfg.BATCH_SIZE,
                                                    train_frac=cfg.TRAIN_TEST_SPLIT)

    model_path = os.path.expanduser(model_name)

    total_records = len(tubgroup.df)
    total_train = int(total_records * cfg.TRAIN_TEST_SPLIT)
    total_val = total_records - total_train
    print('train: %d, validation: %d' % (total_train, total_val))
    steps_per_epoch = total_train // cfg.BATCH_SIZE
    print('steps_per_epoch', steps_per_epoch)

    kl.train(train_gen,
             val_gen,
             saved_model_path=model_path,
             steps=steps_per_epoch,
             train_split=cfg.TRAIN_TEST_SPLIT,
             epochs=1) # <------ Run only 1 epoch


if __name__ == '__main__':
    args = docopt(__doc__)
    cfg = dk.load_config()
    
    if args['train']:
        tub = args['--tub']
        model = args['--model']
        train(cfg, tub, model)
EOF

In [ ]:
# Create a new car using the new template
![ -d ~/d3 ] && rm -rfv ~/d3
!donkey createcar --template robolab --path ~/d3

In [ ]:
%%time

!cat manage.py

# Let's try the new car
%cd ~/d3
!python manage.py train --tub='../SageMaker/tub_8_18-02-09' --model './models/my-test-model'
%cd ~/SageMaker

Sweet, seems to work. Now that we now how to create a new template and our own training part, we can improve the network in any way we like.

## Next

[Porting Keras](./donkey-porting-keras.ipynb)